In [140]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import json
import os
import re
from copy import deepcopy
from pathlib import Path

from aiida import load_profile
from aiida.engine import submit
from aiida.orm import Bool, Dict, Float, Int, StructureData, load_code, load_node
from aiida.plugins import DataFactory
from aiida_quantumespresso.common.types import SpinType
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida.orm.nodes.data.structure import Site
from ase.atoms import Atoms
from ase.visualize import view

load_profile()
import pandas as pd
from aiida.orm import StructureData
from project_settings import project_dir

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

In [ ]:

# ! Currently, for whatever reason, the codes are duplicated
# pw_code = load_code('qe-dev-pw@lumi-small')
# hp_code = load_code('qe-dev-hp@lumi-small')
pw_code = load_code(2182)
hp_code = load_code(2183)

hubbard_data = load_node(3075)
print(hubbard_data.get_ase())
print(hubbard_data.get_quantum_espresso_hubbard_card())

***
## Set up workchain with all of Iurii's settings

In [ ]:
afm_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

scf_dict = afm_builder.scf.pw.parameters.get_dict()
relax_dict = afm_builder.relax.base.pw.parameters.get_dict()
hubbard_dict = afm_builder.hubbard.hp.parameters.get_dict()

# print(json.dumps(scf_dict, sort_keys=False, indent=4))
# print(json.dumps(relax_dict, sort_keys=False, indent=4))
# print(json.dumps(hubbard_dict, sort_keys=False, indent=4))

In [ ]:
# proper_olivine_builder = deepcopy(builder)
# proper_olivine_submit = submit(proper_olivine_builder)
# proper_olivine_submit_pk = proper_olivine_submit.pk
proper_olivine_submit_pk = 4282

In [ ]:
!verdi process status 4282

***
## Change only force convergence threshold in the optimization to see if `reconstruction problem` persists

In [ ]:
only_force_thr_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'only_force_thr.yaml')),
    spin_type=SpinType.COLLINEAR,
)

# only_force_thr_builder = submit(only_force_thr_builder)
# only_force_thr_submit_pk = only_force_thr_builder.pk
# print(only_force_thr_submit_pk)

In [ ]:
# only_force_thr_submit_pk = 4376
!verdi process status 4376

***
## True single-shot (only scf + hp), with default workchain settings

In [ ]:
from aiida.orm import Bool, Int

true_singleshot_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = true_singleshot_builder.pop("relax", None)
true_singleshot_builder.meta_convergence = Bool(False)
true_singleshot_builder.max_iterations = Int(1)

# true_singleshot_submit = submit(true_singleshot_builder)
# print(true_singleshot_submit.pk)



In [ ]:
true_singleshot_submit_pk = 4774
# !verdi process status 4774
true_ss_workchain_node = load_node(true_singleshot_submit_pk)
print(true_ss_workchain_node.mtime - true_ss_workchain_node.ctime)

***
## Default workchain settings, no relaxation


In [ ]:
default_norelax_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

# _ = default_norelax_builder.pop("relax", None)
# default_norelax_submit = submit(default_norelax_builder)
# print(default_norelax_submit.pk)


In [ ]:
default_norelax_submit_pk = 4512
# !verdi process status 4512
default_workchain_node = load_node(default_norelax_submit_pk)
print(default_workchain_node.mtime - default_workchain_node.ctime)

***
## All of Iurii's settings, but no relaxation

In [ ]:
iurii_norelax_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = iurii_norelax_builder.pop("relax", None)
# iurii_norelax_builder = submit(iurii_norelax_builder)
# print(iurii_norelax_builder.pk)

In [ ]:
iurii_norelax_builder_pk = 4486 
# !verdi process status 4486
iurii_workchain_node = load_node(iurii_norelax_builder_pk)
print(iurii_workchain_node.mtime - iurii_workchain_node.ctime)

***
## Compare results of true SS, workchain defaults, and Iurii's settings

In [ ]:
true_ss_output_hubbard = load_node(4813)
default_output_hubbard = load_node(4699)
iurii_output_hubbard = load_node(4822)

In [ ]:
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from pprint import pprint
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
# type(true_ss_output_hubbard)

true_ss_hubbard_array = np.array([el[4] for el in true_ss_output_hubbard.hubbard_parameters])
default_hubbard_array = np.array([el[4] for el in default_output_hubbard.hubbard_parameters])
iurii_hubbard_array = np.array([el[4] for el in iurii_output_hubbard.hubbard_parameters])

pprint(mae(true_ss_hubbard_array, default_hubbard_array))
pprint(mae(true_ss_hubbard_array, iurii_hubbard_array))
pprint(mae(default_hubbard_array, iurii_hubbard_array))

print(true_ss_output_hubbard.get_quantum_espresso_hubbard_card())
print(default_output_hubbard.get_quantum_espresso_hubbard_card())
print(iurii_output_hubbard.get_quantum_espresso_hubbard_card())

pprint(true_ss_hubbard_array)
pprint(default_hubbard_array)
pprint(iurii_hubbard_array)

print(np.abs(true_ss_hubbard_array-iurii_hubbard_array))
print(np.abs(default_hubbard_array-iurii_hubbard_array))


In [ ]:
pprint(true_ss_hubbard_array-default_hubbard_array)
pprint(default_hubbard_array-iurii_hubbard_array)

***
## Set proper AFM magnetic ordering
### Investigate `structure_relabel_kinds` function for trying the relabeling

In [193]:

hubbard_structure = load_node(3075)
hp_workchain = load_node(4560)

hubbard_structure_outside = hp_workchain.outputs.hubbard_structure
hubbard_dict_outside = hp_workchain.outputs.hubbard

def structure_relabel_kinds(hubbard_structure, hubbard_dict, magnetization=None):
    """Create a clone of the given structure but with new kinds, based on the new hubbard sites.

    :param hubbard_structure: ``HubbardStructureData`` instance.
    :param hubbard: the ``hubbard`` output Dict node of a ``HpCalculation``.
    :param magnetization: Dict instance containing the `starting_magnetization` QuantumESPRESSO inputs.
    :return:
    """
    reference = hubbard_structure.clone()
    relabeled = reference.clone()

    relabeled.clear_kinds()
    relabeled.clear_sites()

    kind_suffix = -1
    type_to_kind = {}
    sites = hubbard_structure.sites
    if magnetization is not None:
        old_magnetization = magnetization.get_dict()
        new_magnetization = old_magnetization.copy()
        # Removing old Hubbard spin-polarized atom label.
        for site in hubbard_dict['sites']:
            new_magnetization.pop(site['kind'], None)

    # First do the Hubbard sites, popping the kind name suffix each time a new type is encountered. We do the suffix
    # generation ourselves, because the indexing done by hp.x contains gaps in the sequence.
    relabel_dict = {
        0: 'Mn1',
        1: 'Mn2',
        2: 'Mn1',
        3: 'Mn2',
    }

    # print(type(hubbard_dict['sites']))
    # print(hubbard_dict['sites']) # ! => Returns a list of dicts
    # print(hubbard_structure.sites) # ! => Returns a list of Site objects
    # print(type(hubbard_dict['sites'][0]))
    # print(type(hubbard_structure.sites[0]))
    
    # ! This works now with HubbardStructureData, but ultimately should work with only StructureData
    # ! => Already works with StructureData because it just calls the sites property of the StructureData parent class
    
    final_structuredata = StructureData()
    
    for index, site in enumerate(hubbard_structure.sites):
        # print(index, site)
        raw_site = site.get_raw()
        
        loop_site = deepcopy(site)
        print(type(loop_site))
        loop_site.kind_name = relabel_dict.get(index, site)
        
        print(loop_site)
        print(type(loop_site.kind_name), loop_site.kind_name)
        print(type(loop_site.kind), loop_site.kind)

        # final_structuredata.append_kind(loop_site.kind_name)
        # final_structuredata.append_site(loop_site)
        
        # print(raw_site)
        # print(help(site))
        # ! Return site from raw: Site(raw_site)
        # raise SystemExit
        # symbol = re.search(r'^([A-za-z]+)[0-9]*$', site['kind']).group(1)
        # try:
        #     # We define a `spin_type`, since hp.x does not distinguish new types according to spin
        #     spin_type = str(int(site['new_type']) * int(site['spin']))
        #     kind_name = type_to_kind[spin_type]
        # except KeyError:
        #     kind_suffix += 1
        #     kind_name = f'{symbol}{kind_suffix}'
        #     type_to_kind[spin_type] = kind_name
        #     if magnetization is not None:
        #         # filling 'starting magnetization' with input value but new label;
        #         # if does not present a starting value, pass.
        #         try:
        #             new_magnetization[kind_name] = old_magnetization[site['kind']]
        #         except KeyError:
        #             pass

    # for index, site in enumerate(hubbard_dict['sites']):
    #     print(index, site)
    #     symbol = re.search(r'^([A-za-z]+)[0-9]*$', site['kind']).group(1)
    #     try:
    #         # We define a `spin_type`, since hp.x does not distinguish new types according to spin
    #         spin_type = str(int(site['new_type']) * int(site['spin']))
    #         kind_name = type_to_kind[spin_type]
    #     except KeyError:
    #         kind_suffix += 1
    #         kind_name = f'{symbol}{kind_suffix}'
    #         type_to_kind[spin_type] = kind_name
    #         if magnetization is not None:
    #             # filling 'starting magnetization' with input value but new label;
    #             # if does not present a starting value, pass.
    #             try:
    #                 new_magnetization[kind_name] = old_magnetization[site['kind']]
    #             except KeyError:
    #                 pass

        # loop_site = sites[index]
        # relabeled.append_atom(position=loop_site.position, symbols=symbol, name=kind_name)

    # # Now add the non-Hubbard sites
    # for site in sites[len(relabeled.sites):]:
    #     symbols = hubbard_structure.get_kind(site.kind_name).symbols
    #     names = hubbard_structure.get_kind(site.kind_name).name
    #     relabeled.append_atom(position=site.position, symbols=symbols, name=names)

    # outputs = {'hubbard_structure': relabeled}
    # if magnetization is not None:
    #     outputs.update({'starting_magnetization': Dict(new_magnetization)})

    # return outputs

# print(hubbard_structure.get_quantum_espresso_hubbard_card())


structure_relabel_kinds(
    hubbard_structure=hubbard_structure_outside,
    hubbard_dict=hubbard_dict_outside,
    magnetization=None
)

<class 'aiida.orm.nodes.data.structure.Site'>
kind name 'Mn1' @ -0.015991133342284,0.0,-4.7299542407316
<class 'str'> Mn1


AttributeError: 'Site' object has no attribute 'kind'

In [141]:
# from pymatgen.core.structure import Structure
# from pymatgen.analysis.magnetism.analyzer import CollinearMagneticStructureAnalyzer

# # Read this file: http://webbdcrista1.ehu.es/magndata/index.php?index=0.103
# bulk_structure = Structure.from_file('0.103.mcif')
# magnetic_structure = CollinearMagneticStructureAnalyzer(bulk_structure, make_primitive=False)
# aiidastructure = StructureData(pymatgen=magnetic_structure.get_structure_with_spin())
# aiidastructure.get_kind_names()

In [147]:
from copy import deepcopy
from pymatgen.io.ase import AseAtomsAdaptor

fully_lithiated_df = pd.read_pickle(os.path.join(project_dir, 'data', 'fully_lithiated_df.pkl'))
# fully_lithiated_df['chem_formula'].values
mn_olivine_ase = fully_lithiated_df['ase_in'].values[2]

sorting_dict = {
    'Mn': 0,
    'O': 1,
    'P': 2,
    'Li': 3,
}

# ! Does not take into account initial magmoms for now
mn_olivine_ase_ordered = Atoms(sorted(mn_olivine_ase, key=lambda x: sorting_dict[x.symbol]))
mn_olivine_ase_ordered.set_cell(mn_olivine_ase.get_cell())

mn_olivine_ase_ordered.symbols
mn_olivine_ase_ordered.labels
mn_olivine_ase_ordered.kinds


Symbols('Mn4O16P4Li4')

AttributeError: 'Atoms' object has no attribute 'labels'

In [194]:

# ? Replace Mn with Mn1 and Mn2 as ase or as pmg

# mn_olivine_pmg = AseAtomsAdaptor.get_structure(mn_olivine_ase_ordered)
# mn_olivine_structuredata = StructureData(pymatgen=mn_olivine_pmg)

devel_structuredata = StructureData(ase=mn_olivine_ase_ordered)
devel_pmg = devel_structuredata.get_pymatgen()
devel_ase = devel_structuredata.get_ase()

devel_structuredata.get_kind_names()

new_devel_structuredata = StructureData()
new_devel_structuredata.append_kind('Mn1')

# ! Trying to append special Mn1 and Mn2 labels in ase.Atoms

# ! Trying to append special Mn1 and Mn2 labels in pmg.Structure

# ! Trying to append special Mn1 and Mn2 labels in the structuredata
for isite, site in enumerate(devel_structuredata.sites):
    temp_site = deepcopy(site)
    if isite in (0, 2):
        temp_site.kind_name = 'Mn1'
    elif isite in (1, 3):
        temp_site.kind_name = 'Mn2'
    new_devel_structuredata.append_site(temp_site)


print(devel_structuredata.kinds)
print(devel_structuredata.sites)
print(devel_structuredata.get_site_kindnames())


# mn_olivine_pmg.replace(0, 'Mn1')
# mn_olivine_pmg.replace(1, 'Mn2')
# mn_olivine_pmg.replace(2, 'Mn1')
# mn_olivine_pmg.replace(3, 'Mn2')
# print(mn_olivine_pmg.formula)

# Add magnetization to pmg Structure
# mn_olivine_pmg.add_site_property("magmom", [0.5, 0.5, -0.5, -0.5]+[0]*(mn_olivine_pmg.num_sites-4))


# print(mn_olivine_pmg)

# hubbard_data_inv = HubbardStructureData(structure=mn_olivine_aiida)
# hubbard_data_inv.initialize_onsites_hubbard('Mn', '3d', 4.5618)
# hubbard_data_inv.hubbard_parameters
# hubbard_data_inv.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 


['Mn', 'O', 'P', 'Li']

ValueError: Error using the Kind object. Are you sure it is a Kind object? [Introspection says it is <class 'str'>]

In [ ]:

afm_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_structure,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = afm_builder.pop("relax", None)

# print(type(afm_builder.scf.pw.parameters))

mag_dict = {
    "starting_magnetization": {
        "Mn": 0.5,
        "O": 0.1,
        "P": 0.1,
        "Li": 0.1,
    }
}

afm_builder.scf.pw.parameters["SYSTEM"] = mag_dict
# print(afm_builder.scf.pw.parameters["SYSTEM"])

scf_dict = afm_builder.scf.pw.parameters.get_dict()
hubbard_dict = afm_builder.hubbard.hp.parameters.get_dict()

print(json.dumps(scf_dict, sort_keys=False, indent=4))
print(json.dumps(hubbard_dict, sort_keys=False, indent=4))

# afm_magn_builder = submit(afm_magn_builder)
# only_force_thr_submit_pk = afm_magn_builder.pk
# print(only_force_thr_submit_pk)
# only_force_thr_submit_pk = 4376
# !verdi process report 4376


In [ ]:
hpworkchain_node = load_node(4560)
direct_hubbard = load_node(4655)
print(hpworkchain_node.outputs.hubbard_structure)


# print(type(hubbard_dict_node))


***
## Copied from Marnik's notebook

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the olivine structure!

In [ ]:
hubbard_data_inv = HubbardStructureData(structure=mn_olivine_aiida)

In [ ]:
# hubbard_data.reorder_atoms()

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [ ]:
# Taken from parameters.in of:
# /home/jgeiger/projects/bat_uv_ml/data/olivine_iurii/LixMnPO4/Li1.00/DFT_plus_UV/9_PDOS/LMPO.scf.1.in
hubbard_data_inv.initialize_onsites_hubbard('Mn', '3d', 4.5618)

Here how it is stored in the class.

In [ ]:
hubbard_data_inv.hubbard_parameters

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [ ]:
hubbard_data_inv.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 
# ! Ti has 6 oxygen neighbors in BaTiO3. Similarly, Mn has 6 oxygen neighbors in LMPO (olivine). So, we need to use N+1 in the call to initialize_intersites_hubbard, as the value is used for list slicing, like [:number_of_neighbours], which is exclusive of the last index.

The parameters are saved in the property `hubbard_parameters` as a list.

In [ ]:
hubbard_data_inv.hubbard_parameters

In [ ]:
print(hubbard_data_inv.get_quantum_espresso_hubbard_card())
print(hubbard_data_inv.get_quantum_espresso_hubbard_parameters())

In [ ]:
print(type(mn_olivine_aiida))
print(mn_olivine_aiida.get_ase().get_chemical_symbols())
from aiida.orm.nodes.data.structure import StructureData

In [ ]:
hubbard_data_inv.store()
hubbard_data_inv.pk